# Scoring Function
A scoring function is a function that takes as input a number of objective values and returns an aggregated desirability score.  
The scoring function is dynamically constructed based on a scoring profile, defining the aggregation algorithm, and, for each objective, its desirability function and the required parameters.



## Code Examples

In [1]:
# import the library and print the version
import pumas
print(pumas.__version__)

0.0.0


### Prepare a Scoring Profile


In [ ]:
import json

with open("data/quality_efficiency_cost/scoring_profile.json", 'r') as file:
    scoring_profile_dict = json.load(file)

In [4]:
from pumas.scoring_profile.scoring_profile import ScoringProfile
scoring_profile = ScoringProfile.model_validate(scoring_profile_dict)

ValidationError: 6 validation errors for ScoringProfile
objectives.0.value_type
  Field required [type=missing, input_value={'name': 'quality', 'desi...: 10.0}}, 'weight': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
objectives.0.kind
  Field required [type=missing, input_value={'name': 'quality', 'desi...: 10.0}}, 'weight': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
objectives.1.value_type
  Field required [type=missing, input_value={'name': 'efficiency', 'd...: 10.0}}, 'weight': 2.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
objectives.1.kind
  Field required [type=missing, input_value={'name': 'efficiency', 'd...: 10.0}}, 'weight': 2.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
objectives.2.value_type
  Field required [type=missing, input_value={'name': 'cost', 'desira

### Prepare the data
The data_payload is a dictionary. Each item corresponds to an object to score.
The key of each item is a unique identifier of the object; it will be used to report the results.
The value of each item is a dictionary pf properties, where the key is the property name and the value is the property value.

In [7]:
import json

with open("data/quality_efficiency_cost/data_payload.json", 'r') as file:
    data_payload = json.load(file)


{'A': {'quality': 1.0, 'cost': 30.0, 'efficiency': 0.7}, 'B': {'quality': 5.0, 'cost': 40.0, 'efficiency': 0.2}, 'C': {'quality': 3.0, 'cost': 10.0, 'efficiency': 0.9}, 'D': {'quality': 6.0, 'cost': 60.0, 'efficiency': 0.4}, 'E': {'quality': 2.0, 'cost': 80.0, 'efficiency': 0.5}}


In [6]:
print(f"The type of the data is {type(data_payload)}")
print(f"{data_payload.keys()}")

The type of the data is <class 'dict'>
dict_keys(['A', 'B', 'C', 'D', 'E'])


### Initialize the Scoring Function

In [ ]:
from pumas.scoring_framework.scoring_function import ScoringFunction
scoring_function = ScoringFunction(
        profile=scoring_profile,
    )

### Use the Scoring Function on Data

In [ ]:
result = scoring_function.compute(data=data_payload.get("A"))
print(result)

In [ ]:
results_output = {uid:scoring_function.compute(data=data) for uid, data in data_payload.items()}
from pprint import pprint
pprint(results_output, indent=2)